In [1]:
import gc
import torch
import pandas as pd

from models.load import TrainedModels, get_trained_model
from data.constants import XAMI_MIMIC_PATH, DEFAULT_REFLACX_LABEL_COLS
from utils.coco_utils import get_coco_api_from_dataset

from utils.init import reproducibility, clean_memory_get_device
from data.datasets import ReflacxDataset, collate_fn
from data.transforms import get_transform
from utils.coco_eval import get_eval_params_dict

import pandas as pd
import numpy as np

import utils.print as print_f

from utils.coco_eval import get_eval_params_dict
from utils.engine import xami_evaluate, get_iou_types

from utils.eval import get_ap_ar
from utils.init import reproducibility, clean_memory_get_device
from data.constants import DEFAULT_REFLACX_LABEL_COLS, XAMI_MIMIC_PATH

## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

In [2]:
from enum import Enum

class TrainedModels(Enum):
    CXR_Clinial_fusion1_fusion2="val_ar_0_5436_ap_0_1911_test_ar_0_5476_ap_0_3168_epoch49_WithClincal_05-23-2022 12-06-22_CXR_Clinical_roi_heads_spatialisation"
    CXR_Clinical_fusion1 = "val_ar_0_5476_ap_0_1984_test_ar_0_6038_ap_0_2757_epoch41_WithClincal_05-30-2022 08-01-54_CXR_Clinical_fusion1"
    CXR_Clinical_fusion2= "val_ar_0_4369_ap_0_2098_test_ar_0_4940_ap_0_2218_epoch58_WithClincal_05-30-2022 13-58-43_CXR_Clinical_fusion2"
    CXR="val_ar_0_5659_ap_0_1741_test_ar_0_5390_ap_0_1961_epoch36_WithoutClincal_05-29-2022 12-29-51_CXR"

In [3]:
use_iobb = True
io_type_str = "IoBB" if use_iobb else "IoU"
labels_cols = DEFAULT_REFLACX_LABEL_COLS
iou_thrs = np.array([0.5])


device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [4]:
import os, pickle

model_select = TrainedModels.CXR_Clinial_fusion1_fusion2

with open(os.path.join("training_records", f"{model_select.value}.pkl"), "rb") as f:
    train_info = pickle.load(f)

In [5]:
from models.build import create_model_from_setup

model = create_model_from_setup(labels_cols, train_info.model_setup,
    rpn_nms_thresh=0.3,
    box_detections_per_img=10,
    box_nms_thresh=0.2,
    rpn_score_thresh=0.0,
    box_score_thresh=0.05,
)
model.to(device)

Load custom model
Using pretrained backbone. mobilenet_v3
Using pretrained backbone. mobilenet_v3


MultimodalMaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): Sequential(
    (0): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
    

In [6]:
cp = torch.load(
    os.path.join("trained_models", model_select.value), map_location=device
)


In [7]:
for k in cp['model_state_dict'].keys():
    if "emb" in k:
        print(k)

gender_emb_layer.weight


In [8]:
model, train_info, _, _ = get_trained_model(
    TrainedModels.CXR_Clinial_fusion1_fusion2,
    DEFAULT_REFLACX_LABEL_COLS,
    device,
    rpn_nms_thresh=0.3,
    box_detections_per_img=10,
    box_nms_thresh=0.2,
    rpn_score_thresh=0.0,
    box_score_thresh=0.05,
)

model.to(device)



Load custom model
Using pretrained backbone. mobilenet_v3
Using pretrained backbone. mobilenet_v3
Using SGD as optimizer with lr=0.001


MultimodalMaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): Sequential(
    (0): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
    

In [ ]:
model

MultimodalMaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): Sequential(
    (0): Sequential(
      (0): ConvNormActivation(
        (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (2): Hardswish()
      )
      (1): InvertedResidual(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=16, bias=False)
            (1): BatchNorm2d(16, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1))
    

In [ ]:
train_info.best_ap_val_model_path

'val_ar_0_4491_ap_0_2154_test_ar_0_4870_ap_0_2487_epoch48_WithClincal_05-23-2022 12-04-14_CXR_Clinical_roi_heads_spatialisation'

In [ ]:
use_iobb = True
io_type_str = "IoBB" if use_iobb else "IoU"
labels_cols = DEFAULT_REFLACX_LABEL_COLS
iou_thrs = np.array([0.5])

In [ ]:
model_setup = train_info.model_setup

dataset_params_dict = {
        "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
        "with_clinical": model_setup.use_clinical,
        "dataset_mode": model_setup.dataset_mode,
        "bbox_to_mask": model_setup.use_mask,
        "normalise_clinical_num": model_setup.normalise_clinical_num,
        "labels_cols": labels_cols,
    }


test_dataset = ReflacxDataset(
    **dataset_params_dict, split_str="test", transforms=get_transform(train=False),
)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn,
)

test_coco = get_coco_api_from_dataset(test_dataloader.dataset)

eval_params_dict = get_eval_params_dict(
        test_dataset, iou_thrs=iou_thrs, use_iobb=use_iobb,
    )

iou_types = get_iou_types(model, train_info.model_setup)

# all_range_eval_params_dict = get_eval_params_dict(
#     test_dataset, iou_thrs=full_iou_thrs,
# )


creating index...
index created!
creating index...
index created!


In [ ]:
# model.roi_heads.use_gt_in_train = False
model.roi_heads.score_thresh = 0.3
test_evaluator, test_logger = xami_evaluate(
            model=model,
            data_loader=test_dataloader,
            device=device,
            params_dict=eval_params_dict,
            coco=test_coco,
            iou_types=iou_types,
            score_thres=None,
        )

Evaluation:  [ 0/51]  eta: 0:03:42  loss: 0.0145 (0.0145)  loss_classifier: 0.0075 (0.0075)  loss_box_reg: 0.0000 (0.0000)  loss_objectness: 0.0070 (0.0070)  loss_rpn_box_reg: 0.0000 (0.0000)  model_time: 3.9707 (3.9707)  evaluator_time: 0.0010 (0.0010)  time: 4.3545  data: 0.3305  max mem: 516
Evaluation:  [50/51]  eta: 0:00:00  loss: 0.0367 (0.0602)  loss_classifier: 0.0115 (0.0206)  loss_box_reg: 0.0073 (0.0155)  loss_objectness: 0.0138 (0.0212)  loss_rpn_box_reg: 0.0016 (0.0029)  model_time: 0.0512 (0.1326)  evaluator_time: 0.0010 (0.0007)  time: 0.3999  data: 0.3060  max mem: 528
Evaluation: Total time: 0:00:25 (0.4929 s / it)
Averaged stats: loss: 0.0367 (0.0602)  loss_classifier: 0.0115 (0.0206)  loss_box_reg: 0.0073 (0.0155)  loss_objectness: 0.0138 (0.0212)  loss_rpn_box_reg: 0.0016 (0.0029)  model_time: 0.0512 (0.1326)  evaluator_time: 0.0010 (0.0007)
Accumulating evaluation results...
DONE (t=0.00s).
IoU metric: bbox
 Average Precision  (AP) @[ IoBB=0.50:0.50 | area=   all |

In [ ]:
test_ap_ar = get_ap_ar(
    test_evaluator, areaRng="all", iouThr=0.5, maxDets=10,
)

test_ap_ar

{'ap': 0.19869556974093783, 'ar': 0.2787301587301587}